# Tutorial 4 — Gaussian model for neural population activity

**Author:** [Vito Dichio](https://sites.google.com/view/vito-dichio/home),
Fellow in AI, ENS–PSL, Paris — ✉️ vito.dichio@psl.eu

**Course:** *Machine Learning: Theory and Applications*
Master in Cognitive Sciences, ENS–PSL — Fall 2025/26 (Lecturer: Simona Cocco)

**Format:** Practical Session (TD)

#### Bibliography:
[1] Cocco et al., *From Statistical Physics to Data-Driven Modelling: with Applications to Quantitative Biology*, Oxford University Press (2022)

 #### 🎯 Goals

**Goals**
- Build and preprocess spike-count matrices from neural recordings
- Fit an **independent Bernoulli model** to population activity during the task
- Evaluate the likelihood of neural configurations during **pre-** and **post-task** sleep
- Extend to a **Gaussian model** capturing pairwise correlations
- Compare inferred coupling (precision) matrices across behavioral states

**Note** / This notebook is builds on the *Tutorial 3: PCA and replay of neural activity during sleep* of this course. See there for more details on the dataset and references.

## Context and experimental data (reminder)

We consider again the dataset introduced in **Tutorial 3**, which records **neural population activity** from the **prefrontal cortex (PFC)** of a rat across three behavioral epochs:

1. **Sleep 1 (pre-task sleep)** – baseline neural activity before the task
2. **Maze (task)** – neural activity during a spatial navigation task
3. **Sleep 2 (post-task sleep)** – neural activity after task completion

The data come from **multi-electrode recordings** capturing the spiking activity of multiple neurons across these behavioral states. The central question is whether **task-related neural patterns** are **reactivated during post-task sleep**, a hallmark of **replay** and **memory consolidation** within the two-stage model of learning.

We denote by $X_{\text{pre}}$, $X_{\text{task}}$, and $X_{\text{post}}$, the spike times are expressed in seconds, starting from $t = 0$, as obtained in Tutorial 3.


## 1 - From spikes to binary activity

#### 🎯 Question 1a. Discretization

- For each condition (`pre`, `task`, and `post`), and using time bins of width $\Delta t = 0.05\,\mathrm{s}$ (50 ms), construct the spike-count matrices $ S \in \mathbb{N}^{M \times L}, $
where each **row** corresponds to a time bin and each **column** to a neuron. Each entry $S_{b,i}$ represents the **number of spikes** emitted by neuron $i$ within time bin $b$.

#### 🎯 Question 1b. Binarisation
Then,  for each condition, **binarize** the data to obtain
$$
\sigma_{b,i} =
\begin{cases}
1 & \text{if}\ S_{b,i} >0 \\
0 & \text{otherwise.}
\end{cases}\ .
$$
Note that the binarized activity matrix $\boldsymbol{\sigma} \in \{0,1\}^{M \times L}$ has the same dimensions as $S$.

We will use the symbol $\boldsymbol{\sigma}_{\bullet,i} \in \{0,1\}^M$ to denote the activity of neuron $i$ across all $M$ time bins, i.e. the $i$-th column of $\sigma$. Similarly, $\boldsymbol{\sigma}_{b,\bullet} \in \{0,1\}^L$ denote the population activity configuration in time bin $b$, i.e. the $b$-th row of $\sigma$.


## 2 - Independent-site model

In this section, we introduce a simple **independent model** of neural population activity, where each neuron is represented as a binary random variable that can either fire ($\sigma_{b,i}=1$) or remain silent ($\sigma_{b,i}=0$) in each time bin, **independently** of all others
(*cf.* Section 5.2.1 in [1]).

This model neglects correlations between neurons but provides a useful **baseline**: it captures overall firing propensities and allows us to test whether population activity during **pre-task** and **post-task sleep** becomes more or less compatible with the statistics learned during the **task**.

Let $\boldsymbol{\sigma}_{b,\bullet} = (\sigma_{b,1}, \dots, \sigma_{b,L})$ denote the **population activity vector** at time bin $b$. The model assumes that for all $b$, the joint probability factorizes as

$$
P^{\text{ind}}(\boldsymbol{\sigma}_{b,\bullet})
= \prod_{i=1}^L P_i(\sigma_{b,i})
\quad \text{with} \quad
P_i(\sigma_{b,i}) = \frac{e^{h_i \sigma_{b,i}}}{1 + e^{h_i}}.
$$

The MLE estimates of the parameters $h_i$ can be obtained by the moment-matching condition:
$$
\langle \sigma_i \rangle_{\text{data}} = \frac{e^{h_i}}{1 + e^{h_i}}\ ,
$$
where $\langle \sigma_i \rangle_{\text{data}}$ are the empirical mean activities of each neuron $i$:
$$
\langle \sigma_i \rangle_{\text{data}} = \frac{1}{M} \sum_{b=1}^M \sigma_{b,i}
$$


#### 🎯 Question 2a. Fitting the model

Starting grom the binarized data matrices from (1.b), estimate the empirical mean activities $\langle \sigma_i \rangle_{\text{data}}$ for each neuron $i$ from the **task** session, and use the moment-matching condition to derive the corresponding fields $h_i$.

#### 🎯 Question 2b. Scoring configurations with the task model

Using the independent model fitted on the **task** data, the **log-probability** of each population activity configuration $\boldsymbol{\sigma}_{b,\bullet}$ reads:
$$
\log P^{\text{ind}}(\boldsymbol{\sigma}_{b,\bullet}) = \sum_{i=1}^{L} \left[ \sigma_{b,i} \log p_i + (1 - \sigma_{b,i})\log(1 - p_i) \right],
$$
where $p_i = \frac{e^{h_i}}{1 + e^{h_i}}$ are the firing probabilities from the task session.

**2.b.i** / Compute the log-probability for each time bin $b$ in the **pre-task**, **task**, and **post-task** sessions. Report the **mean and standard deviation** of log-probabilities for each session.

**2.b.ii** / Plot and compare the **distributions** of log-probability scores (one for each time bin) across the three sessions.

**2.b.iii** / Interpretation: What does the comparison of these distributions tell you about the similarity of population activity patterns before, during, and after the task? Which session's activity is best captured by the task model?

#### 🎯 Question 2c. Comparing neural representations across behavioral states

In Question 2b, you evaluated how well the **task model** scores configurations from different sessions. Now, we ask a complementary question: how do the **learned parameters** (the fields $h_i$) themselves differ across behavioral states?

**2.c.i** / Fit independent models separately on the **pre-task** and **post-task** data to obtain field vectors $\mathbf{h}_{\text{pre}}$ and $\mathbf{h}_{\text{post}}$.

**2.c.ii** /   Calculate and report the **cosine similarity** for the following pairs of local field vectors: $(\mathbf{h}_{\text{pre}}, \mathbf{h}_{\text{task}})$, $(\mathbf{h}_{\text{post}}, \mathbf{h}_{\text{task}})$,  $(\mathbf{h}_{\text{post}}, \mathbf{h}_{\text{pre}})$.

ℹ️ **Reminder**: The cosine similarity measures the alignment between two vectors, ranging from $-1$ (opposite directions) to $+1$ (same direction), and is defined as:
  $$
  \text{cosine similarity}(v_1, v_2) = \frac{v_1 \cdot v_2}{\|v_1\| \, \|v_2\|}
  $$
  where $v_1 \cdot v_2 $ is the dot product and $\|v\| $ is the Euclidean norm.



**2.c.iii** / Interpretation: Does the neural representation during post-task sleep resemble the task representation more closely than pre-task sleep does? How do these findings compare with those from Question 2b?

## 3 - Gaussian model with pairwise correlations

The independent model captures individual firing propensities but ignores **pairwise correlations** between neurons. We now extend our analysis to a **Gaussian model** that accounts for these correlations, providing a richer description of population activity.  (*cf.* Section 5.1 in [1])

For z-scored neural activity $Y_{b,\bullet} = (Y_{b,1}, \dots, Y_{b,L})$ with zero mean and unit variance, the Gaussian model assumes:
$$
P^{\text{Gauss}}(Y_{b,\bullet}) = \frac{1}{(2\pi)^{L/2} |C|^{1/2}} \exp\left(-\frac{1}{2} Y_{b,\bullet}^T \, T \, Y_{b,\bullet}\right),
$$

where $T = C^{-1}$ is the **precision matrix** (the inverse of the covariance matrix $C$).

Because the data are z-scored (zero mean, unit variance), the model is fully characterized by the **off-diagonal elements** $T_{ij}$ ($i \neq j$), which encode **effective pairwise couplings** between neurons. The **maximum likelihood estimate (MLE)** of the precision matrix is obtained by inverting the empirical covariance matrix: $T = C^{-1}$. A large positive (negative) coupling $T_{ij}$ indicates that neurons $i$ and $j$ tend to be active together (in opposite patterns) more often than expected by chance.

#### 🎯 Question 3a. Data preprocessing and correlation structure

Starting from the **spike-count matrices** $S_{\text{pre}}$, $S_{\text{task}}$, and $S_{\text{post}}$ (not the binarized versions!), **z-score** each matrix to obtain standardized activity matrices $Y$ with zero mean and unit variance per neuron, and compute the **sample correlation matrices** for each session:
$$
C = \frac{1}{M} Y^T Y
$$

*Note: See Tutorial 3, Q1b-c for reference on z-scoring the spike-count matrices (not the binarized ones!) and correlation computation.*


#### 🎯 Question 3b. Scoring configurations with the Gaussian model

**3.b.i** / Compute the **precision matrix** $T_{\text{task}} = C_{\text{task}}^{-1}$ from the task session.

**3.b.ii** / For each time bin $b$ in all three sessions, compute the log-probability under the Gaussian model fitted on the task:
  $$
  \log P^{\text{Gauss}}(Y_{b,\bullet}) = -\frac{1}{2} \sum_{i \neq j} T_{ij} Y_{b,i} Y_{b,j}
  $$
  **Note:** We exclude diagonal terms $T_{ii}$ since the z-scoring already accounts for individual firing rates. Furthermore, the normalization constant is omitted as it is the same for all configurations.

Report the **mean and standard deviation** of log-probability scores for each session.

**3.b.iii** / Create visualizations to compare the distributions.
- Plot histograms of log-probability scores for all three sessions (use logarithmic y-scale if needed).
- Plot log-probability scores as a function of time bin for each session.

**3.b.iv** / Intepretation: How do these results compare with those from the independent model (Question 2b)? Does accounting for correlations change which session's activity is best captured by the task model?

#### 🎯 Question 3c. Comparing coupling matrices across behavioral states

The precision matrix $T$ (or equivalently, its negative $J = -T$, called the **coupling matrix**) encodes the interaction structure between neurons. We now ask: how does this interaction structure differ across behavioral states?

**3.c.i** / Compute the coupling matrices for all three sessions:
  $$
  J_{\text{pre}} = -C_{\text{pre}}^{-1}, \quad J_{\text{task}} = -C_{\text{task}}^{-1}, \quad J_{\text{post}} = -C_{\text{post}}^{-1}
  $$

**3.c.ii** / Visualize the three coupling matrices as heatmaps (with diagonal elements set to zero for clarity).

**3.c.iii** / Compute the **cosine similarity** (see definition above) between the linearised off-diagonal elements of each pair of coupling matrices: ($J_{\text{pre}}$ and $J_{\text{task}}$), ($J_{\text{post}}$ and $J_{\text{task}}$), and ($J_{\text{pre}}$ and $J_{\text{post}}$).

💡 *Hint*: You can extract and vectorize the off-diagonal elements by using the following helper function:

In [6]:
import numpy as np

def offdiag_vec(J):
    """Vectorize the strictly upper-triangular (off-diagonal) entries of J."""
    iu = np.triu_indices_from(J, k=1)
    return J[iu]

# Example:
#J_example = np.array([[1, 2, 3],  [4, 5, 6], [7, 8, 9]]) # 3x3 example matrix
#vec = offdiag_vec(J_example)
#print(J_example)
#print('')
#print(vec)

**3.c.iv** / Interpretation: Do the pairwise coupling patterns during post-task sleep resemble those during the task more than pre-task sleep does? What does this tell you about the consolidation of interaction structure between neurons during sleep?